In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

In [3]:
import urllib.request
# Download sample file
model_url = "https://dl.dropboxusercontent.com/scl/fi/lzv3ba5paum6srhte4z2t/GSM3271406_pbmc_18.impute.3dg.txt?rlkey=dc7k1gg5ghv2v7dsl0gg1uoo9&dl=0"

file_content = ''
with urllib.request.urlopen(model_url) as f:
    res = f.read().decode('utf-8')
    file_content = res

len(file_content)

3479387

In [14]:
x_column = []
y_column = []
z_column = []
chrom_column = []
coord_column = []
for line in file_content.splitlines():
    # print(line)
    tokens = line.split('\t')
    x = float(tokens[2])
    y = float(tokens[3])
    z = float(tokens[4])
    chrom = tokens[0]
    coord = int(tokens[1])
    
    x_column.append(x)
    y_column.append(y)
    z_column.append(z)
    chrom_column.append(chrom)
    coord_column.append(coord)

print(len(x_array))
print(len(coord_array))

# converting to arrow arrays
x_array = pa.array(x_column)
y_array = pa.array(y_column)
z_array = pa.array(z_column)
chrom_array = pa.array(chrom_column)
coord_array = pa.array(coord_column)

model_table = pa.Table.from_arrays([x_array, y_array, z_array, chrom_array, coord_array], names=["x", "y", "z", "chr", "coord"])
pq.write_table(model_table, 'model.parquet', compression=None)

schema = pa.schema([
    pa.field('x', x_array.type),
    pa.field('y', y_array.type),
    pa.field('z', z_array.type),
    pa.field('chr', chrom_array.type),
    pa.field('coord', coord_array.type)
])

with pa.OSFile('model.arrow', 'wb') as sink:
    with pa.ipc.new_file(sink, schema=schema) as writer:
        batch = pa.record_batch([x_array, y_array, z_array, chrom_array, coord_array], schema=schema)
        writer.write(batch)

57990
57990
